In [ ]:
import optuna
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, confusion_matrix, roc_curve, auc

## Model Training

In this notebook we will be looking to train the XGBoost model and see how good our features are for the model, checking statistics like accuracy, F-1 score, etc.

First, let's separate our target and our features.

In [1]:
# Load the teams data
dataset = pd.read_csv('./cleaned_data/dataset.csv')

label = 'PlayoffNextSeason'
features = [
    'CumulativePlayoffProgScore', '3P%', 'FT%',
    'OFFRTG', 'DEFRTG', 'AST RATIO',
    'REB%', 'TOV%', 'PACE', 'AvgPIE_NextYearPlayers', 'Performance_NextYearCoach'
]

# Prepare feature and label data
X = dataset[features]
y = dataset[label]

NameError: name 'pd' is not defined

We then plotted a correlation matrix and mutual information classification table to distinguish our most important features.

In [ ]:
# ============================
# Display Initial Correlation Matrix
# ============================

# Calculate and plot the correlation matrix
correlation_matrix = dataset[features + [label]].corr()
plt.figure(figsize=(20, 18))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix (Before Selection)')
plt.show()

In [ ]:
# ============================
# Display Initial Mutual Information Scores
# ============================

# Calculate mutual information
mi_scores = mutual_info_classif(X, y, discrete_features='auto', random_state=42)
mi_scores = pd.Series(mi_scores, index=X.columns, name='MI Scores')
mi_scores = mi_scores.sort_values(ascending=False)

plt.figure(figsize=(10, 8))
sns.barplot(x=mi_scores.values, y=mi_scores.index)
plt.title('Mutual Information Scores (Before Selection)')
plt.xlabel('Mutual Information')
plt.ylabel('Features')
plt.show()

Then we identified which features had a correlation above 0.9 with the target variable.

In [ ]:
'''
# ============================
# Correlation Filtering
# ============================

# Calculate the correlation matrix
correlation_matrix = X.corr()
correlation_threshold = 0.9
correlated_features = set()

# Identify features with correlation above the threshold
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            correlated_features.add(correlation_matrix.columns[i])

# Keep only non-correlated features
X = X.drop(columns=correlated_features)
print("Remaining features correlation filtering:", list(X))
'''

We also filtered the features by checking mutual information scores above a certain threshold.

In [ ]:
'''
# ============================
# Mutual Information Filtering
# ============================

# Calculate mutual information
mutual_info = mutual_info_classif(X, y, discrete_features='auto', random_state=42)
mutual_info_series = pd.Series(mutual_info, index=X.columns)

# Set a threshold for mutual information
mi_threshold = 0.02
selected_features_mi = mutual_info_series[mutual_info_series > mi_threshold].index.tolist()

# Filter the dataset based on selected features
X = X[selected_features_mi]
print("Remaining features after mutual information filtering:", selected_features_mi)
'''

#### Defining the Model

In [ ]:
# ============================
# Define the XGBoost model
# ============================

def create_model(trial):
    return XGBClassifier(
        random_state=42,
        eval_metric='logloss',
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        n_estimators=trial.suggest_int('n_estimators', 50, 500),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        min_child_weight=trial.suggest_int('min_child_weight', 1, 10),
        subsample=trial.suggest_float('subsample', 0.5, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1.0),
        gamma=trial.suggest_float('gamma', 0, 5),
        reg_alpha=trial.suggest_float('reg_alpha', 0, 1),
        reg_lambda=trial.suggest_float('reg_lambda', 0, 1),
        scale_pos_weight=trial.suggest_float('scale_pos_weight', 1, 100)
    )

After defining the model, `StratifiedKFold` with K=10 was used to mantain class distribution in each fold. We also defined a custom scorer for specificity, which is the true negative rate. This ensures that the cross-validation process is robust.

In [ ]:
# Use StratifiedKFold to maintain class distribution in each fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Define specificity scorer
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

specificity_scorer = make_scorer(specificity)

# Define the scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'specificity': specificity_scorer
}

#### Hyperparameter Tuning

`Optuna` was used in order to find the best hyperparameters for this model. An objective function was created to combine mean accuracy and mean AUC to evaluate the model's performance, and StratifiedKFold cross-validation to do the same.

In [2]:
# ============================
# Hyperparameter Tuning with Optuna
# ============================

def objective(trial):
    model = create_model(trial)
    cv_results = cross_validate(model, X, y, cv=skf, scoring=['accuracy', 'roc_auc'], n_jobs=-1)
    mean_accuracy = np.mean(cv_results['test_accuracy'])
    mean_auc = np.mean(cv_results['test_roc_auc'])

    return 0.5 * mean_accuracy + 0.5 * mean_auc

sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=1000, n_jobs=-1)

# Use the best hyperparameters to create the final model
best_params = study.best_params
print("Best Hyperparameters:", best_params)
print("Best Value:", study.best_value)
final_model = XGBClassifier(**best_params, random_state=42, eval_metric='logloss')
#final_model = XGBClassifier(
#    learning_rate=0.1,       # Step size shrinkage
#    n_estimators=100,        # Number of trees
#    max_depth=6,             # Maximum depth of trees
#    min_child_weight=1,      # Minimum sum of instance weight (hessian) in a child
#    subsample=0.8,           # Subsample ratio of training data
#    colsample_bytree=0.8,    # Subsample ratio of features per tree
#    gamma=0,                 # Minimum loss reduction to make a split
#    reg_alpha=0,             # L1 regularization
#    reg_lambda=1,            # L2 regularization
#    scale_pos_weight=1,      # Used for imbalanced datasets; adjust if needed
#    random_state=42,         # Ensures reproducibility
#    verbosity=1              # Controls the output verbosity
#)

NameError: name 'optuna' is not defined

#### Results

Finally, we evaluated the model using cross-validation, obtaining important performance metrics and feature importance. The multiple performance metrics were displayed and ROC curves were plotted so we could analyse our results and see what could be improved.

In [ ]:
# ============================
# Final Evaluation
# ============================
cv_results = cross_validate(
    final_model, X, y, cv=skf, scoring=scoring, return_estimator=True, n_jobs=-1
)

feature_importances = []
for estimator in cv_results['estimator']:
    feature_importances.append(estimator.feature_importances_)

average_importances = np.mean(feature_importances, axis=0)

feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Average Importance': average_importances
}).sort_values(by='Average Importance', ascending=False)

# Display the feature importance table
display(importance_df)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Average Importance'])
plt.xlabel('Average Importance')
plt.ylabel('Feature')
plt.title('Average Feature Importance Across Folds')
plt.gca().invert_yaxis()
plt.show()

print("Mean Cross-Validation Accuracy:", cv_results['test_accuracy'].mean())
print("Mean Cross-Validation Precision:", cv_results['test_precision'].mean())
print("Mean Cross-Validation Recall:", cv_results['test_recall'].mean())
print("Mean Cross-Validation F1 Score:", cv_results['test_f1'].mean())
print("Mean Cross-Validation AUC Score:", cv_results['test_roc_auc'].mean())
print("Mean Cross-Validation Specificity:", cv_results['test_specificity'].mean())

# ROC Curve Visualization with Individual Folds
plt.figure(figsize=(10, 8))
mean_fpr = np.linspace(0, 1, 100)
tprs = []
aucs = []
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    final_model.fit(X_train, y_train)
    y_proba = final_model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label=f'Fold {i+1} (AUC = {roc_auc:.2f})')

# Plot the mean ROC curve
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='b', label=f'Mean ROC (AUC = {mean_auc:.2f})', lw=2)

# Plot the random chance line
plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random Chance')

plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('ROC Curves with Individual Folds and Mean')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()